In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open("./data/shakespeare.txt", "r") as f:
    text = f.read()

In [93]:
encoder = {char: idx for idx, char in enumerate(sorted(set(text)))}
decoder = {idx: char for idx, char in enumerate(sorted(set(text)))}
encode = lambda x: [encoder[char] for char in x]
decode = lambda x: ''.join([decoder[idx] for idx in x])

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape)
print(data.dtype)
n = 0.9
train_data = data[:int(n * len(data))]
val_data = data[int(n * len(data)):]

torch.Size([1115394])
torch.int64


In [84]:
print(encode(text[:10]))
print(decode(encode(text[:10])))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
First Citi


In [68]:

train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [70]:
xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context}, target is {target}")


torch.Size([4, 8])
tensor([[ 1, 15, 50, 39, 59, 42, 47, 53],
        [42, 11,  0, 13, 52, 42,  1, 63],
        [43,  1, 41, 53, 59, 52, 57, 43],
        [57, 43, 57,  1, 61, 43, 43, 54]])
torch.Size([4, 8])
tensor([[15, 50, 39, 59, 42, 47, 53,  1],
        [11,  0, 13, 52, 42,  1, 63, 53],
        [ 1, 41, 53, 59, 52, 57, 43, 50],
        [43, 57,  1, 61, 43, 43, 54, 47]])
When input is tensor([1]), target is 15
When input is tensor([ 1, 15]), target is 50
When input is tensor([ 1, 15, 50]), target is 39
When input is tensor([ 1, 15, 50, 39]), target is 59
When input is tensor([ 1, 15, 50, 39, 59]), target is 42
When input is tensor([ 1, 15, 50, 39, 59, 42]), target is 47
When input is tensor([ 1, 15, 50, 39, 59, 42, 47]), target is 53
When input is tensor([ 1, 15, 50, 39, 59, 42, 47, 53]), target is 1
When input is tensor([42]), target is 11
When input is tensor([42, 11]), target is 0
When input is tensor([42, 11,  0]), target is 13
When input is tensor([42, 11,  0, 13]), target is 52

In [147]:
torch.manual_seed(1337)

class BiGramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(BiGramLanguageModel, self).__init__()
        # Each token reads off the logits for the next token from a lookup table
        self.embeddings = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.embeddings(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [148]:
m = BiGramLanguageModel(vocab_size=65)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.9837, grad_fn=<NllLossBackward0>)
<class 'list'>

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [150]:
optimizer = torch.optim.Adam(m.parameters(), lr = 1e-3)

In [156]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch("train")
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(f"Step {steps}, loss {loss}")


Step 9999, loss 2.4670114517211914


In [164]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # Batch, Time, Channel (features)
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


In [175]:
xbow = torch.zeros((B,T,C))
print(xbow.shape)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, dim=0)

torch.Size([4, 8, 2])


In [178]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [179]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [191]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / a.sum(dim=1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b

In [192]:
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [193]:
b

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

In [194]:
c

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])

In [205]:
wei = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(wei == 0, float('-inf'))
wei = F.softmax(wei,dim=1)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [201]:
xbow2 = wei @ x # (B,T,T) * (B,T,C) -> (B,T,C)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [216]:
from httpx import head


torch.manual_seed(1337)
B,T,C = 4,8,32 # Batch, Time, Channel (features)
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) # (B,T,H) @ (B,H,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim=-1)
v = value(x)
out = wei @ v
# out = wei @ x
print(out.shape)

torch.Size([4, 8, 16])


In [212]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [215]:
value(x)

tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9067e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8599e-01,  2.8623e-01,
           5.7099e-01],
         [ 8.3212e-01, -8.1437e-01, -3.2425e-01,  5.1913e-01, -1.2520e-01,
          -4.8982e-01, -5.2867e-01, -3.1393e-02,  1.0723e-01,  8.2692e-01,
           8.1322e-01, -2.7132e-02,  4.7754e-01,  4.9801e-01, -1.3769e-01,
           1.4025e+00],
         [ 6.0346e-01, -2.4995e-01, -6.1588e-01,  4.0678e-01,  3.3283e-01,
          -3.9097e-01,  1.3119e-01,  2.1718e-01, -1.2991e-01, -8.8281e-01,
           1.7242e-01,  4.6522e-01, -4.2710e-01, -7.6754e-02, -2.8524e-01,
           1.3875e+00],
         [ 6.6568e-01, -7.0960e-01, -6.0986e-01,  4.3484e-01,  8.9754e-01,
          -9.2983e-01,  6.8324e-02,  1.8632e-01,  5.4002e-01,  2.4271e-01,
          -6.9225e-01,  4.9774e-01,  4.8503e-01,  6.6076e-01,  8.7671e-01,
           7.4567e-02],
    